In [1]:
import pandas as pd

from datasets import load_dataset

dataset = load_dataset("heegyu/news-category-dataset")

Generating train split:   0%|          | 0/209527 [00:00<?, ? examples/s]

In [10]:
def remove_user_and_links(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
    
def lower_case(text):
    text = text.lower()
    return text
    
def preprocess_text(text):
    text = lower_case(text)
    text = remove_user_and_links(text)
    return text

In [19]:
df = dataset['train'].to_pandas()[['headline', 'category', 'short_description']]

In [20]:
df = df[(df['category'] == 'WORLD NEWS') | (df['category'] == 'POLITICS') | (df['category'] == 'U.S. NEWS') | (df['category'] =='SPORTS') | (df['category'] == 'BUSINESS') | (df['category'] == 'SCIENCE') | (df['category'] == 'TECH')]

Consists of class ids 1-4 where 1-World, 2-Sports, 3-Business, 4-Sci/Tech 

In [21]:
df['text'] = df['headline'] + ' ' + df['short_description']

df['text'] = df['text'].apply(preprocess_text)

df = df[['text', 'category']]

In [22]:
df.loc[df['category'].isin(['WORLD NEWS', 'POLITICS', 'U.S. NEWS']), 'category'] = 0
df.loc[df['category'].isin(['SPORTS']), 'category'] = 1
df.loc[df['category'].isin(['BUSINESS']), 'category'] = 2
df.loc[df['category'].isin(['SCIENCE', 'TECH']), 'category'] = 3

In [25]:
df = df.rename(columns={'category': 'label'})

In [30]:
from sklearn.model_selection import train_test_split
df_t, df_tes  = train_test_split(df, test_size=0.2, random_state=42)

In [34]:
df_t.to_csv('train.csv', index=False)
df_tes.to_csv('test.csv', index=False)